In [13]:
data_directory = "data/gamutrf/gamutrf-sd-gr-ieee-wifi/test_offline/"

# I/Q dataset paramters
num_iq_samples = 1024
only_use_start_of_burst = True

# Spectrogram parameters
n_fft = 1024
time_dim = 512
n_samples = n_fft * time_dim
yolo_class_list = ["wifi", "anom_wifi"]

# I/Q dataset

In [14]:
import numpy as np
import torchsig.transforms as ST
from sigmf_pytorch_dataset import SigMFDataset

transform = ST.Compose([
    ST.Normalize(norm=np.inf),
    ST.ComplexTo2D(),
])

dataset = SigMFDataset( 
    root=data_directory, 
    sample_count= num_iq_samples, 
    transform = transform,
    class_list = yolo_class_list,
    only_first_samples=only_use_start_of_burst,
)

class_counts = {dataset.class_list[k]:v for k,v in dataset.get_class_counts().items()}
print("\nDataset info:")
print(f"{len(dataset)=}, {class_counts=}")

Class List: ['wifi', 'anom_wifi']

Dataset info:
len(dataset)=1807, class_counts={'wifi': 903, 'anom_wifi': 904}


# Spectrogram dataset

In [10]:
import glob
import yaml

from pathlib import Path
from tqdm import tqdm

from data import *

# Convert SigMF to spectrograms and YOLO labels (only needs to run once)

In [5]:
labels = set()
image_dirs = []

for f in tqdm(glob.glob(str(Path(data_directory,"*-meta")))):
    
    d = Data(f)
    d.generate_spectrograms(n_samples, n_fft, cmap_str="turbo", overwrite=False)
    yolo_label_outdir = str(Path(data_directory,"yolo","labels"))
    yolo_image_outdir = str(Path(data_directory,"yolo","images"))
    image_dirs.append(yolo_image_outdir)
    d.export_yolo(yolo_label_outdir, image_outdir=yolo_image_outdir, yolo_class_list=yolo_class_list)

  0%|                                                                                    | 0/2 [00:00<?, ?it/s]


Loaded 
 Data file: data/gamutrf/gamutrf-sd-gr-ieee-wifi/test_offline/anomwifi-encoding-0-sendsps-10000000.s16.zst.sigmf-data 
 SigMF-Meta file: data/gamutrf/gamutrf-sd-gr-ieee-wifi/test_offline/anomwifi-encoding-0-sendsps-10000000.s16.zst.sigmf-meta

Saving data/gamutrf/gamutrf-sd-gr-ieee-wifi/test_offline/anomwifi-encoding-0-sendsps-10000000.s16.zst.sigmf-meta

Writing YOLO txt label files



 50%|██████████████████████████████████████                                      | 1/2 [00:04<00:04,  4.28s/it]

Saving data/gamutrf/gamutrf-sd-gr-ieee-wifi/test_offline/anomwifi-encoding-0-sendsps-10000000.s16.zst.sigmf-meta


Loaded 
 Data file: data/gamutrf/gamutrf-sd-gr-ieee-wifi/test_offline/wifi-encoding-0-sendsps-10000000.s16.zst.sigmf-data 
 SigMF-Meta file: data/gamutrf/gamutrf-sd-gr-ieee-wifi/test_offline/wifi-encoding-0-sendsps-10000000.s16.zst.sigmf-meta

Saving data/gamutrf/gamutrf-sd-gr-ieee-wifi/test_offline/wifi-encoding-0-sendsps-10000000.s16.zst.sigmf-meta

Writing YOLO txt label files



100%|████████████████████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.27s/it]

Saving data/gamutrf/gamutrf-sd-gr-ieee-wifi/test_offline/wifi-encoding-0-sendsps-10000000.s16.zst.sigmf-meta



# Creat YOLOv8 data.yaml

In [11]:
image_dirs = list(set(image_dirs))


yolo_yaml = {}
yolo_yaml["train"] = image_dirs[0]
yolo_yaml["names"] = yolo_class_list
yolo_yaml["nc"] = len(yolo_class_list)

with open('data.yml', 'w') as outfile:
    yaml.dump(yolo_yaml, outfile, default_flow_style=False)